In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.utils.data_utils import get_file
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, Flatten, BatchNormalization
from keras.layers import SimpleRNN
from keras.layers.recurrent import LSTM
from keras.layers.merge import Add
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop, Adam

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
# Sometimes it's useful to have a zero value in the dataset, e.g. for padding
chars.insert(0, '\0')
vocab_size = len(chars)

print('vocab size:', vocab_size)
''.join(chars)

vocab size: 85


'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆäæéë'

In [4]:
char_to_idx = dict((c, i) for i, c in enumerate(chars))
idx_to_char = dict((i, c) for i, c in enumerate(chars))

In [5]:
idx = [char_to_idx[c] for c in text]

## 根据3个char 得到第4个char

In [6]:
clen = 3

c1_data = [idx[i] for i in range(0, len(idx)-clen-1, clen)] # 这里的间距调成1应该也是没问题的
c2_data = [idx[i+1] for i in range(0, len(idx)-clen-1, clen)]
c3_data = [idx[i+2] for i in range(0, len(idx)-clen-1, clen)]
c4_data = [idx[i+3] for i in range(0, len(idx)-clen-1, clen)]

In [7]:
x1 = np.array(c1_data)
x2 = np.array(c2_data)
x3 = np.array(c3_data)
y = np.array(c4_data)

In [8]:
x1.shape, y.shape

((200297,), (200297,))

embedding layers

In [9]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [10]:
n_fac = 42 # 隐藏因子数
c1_inp, c1_emb = embedding_input('c1', vocab_size, n_fac)
c2_inp, c2_emb = embedding_input('c2', vocab_size, n_fac)
c3_inp, c3_emb = embedding_input('c3', vocab_size, n_fac)

hidden layers

In [11]:
n_hidden = 256

dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='tanh')

In [12]:
c1_den = dense_in(c1_emb)

c2_den = dense_in(c2_emb)
hidden_2 = dense_hidden(c1_den)
hidden_2_add = Add()([c2_den, hidden_2])

c3_den = dense_in(c3_emb)
hidden_3 = dense_hidden(hidden_2_add)
hidden_3_add = Add()([c3_den, hidden_3])


In [13]:
c4_out = Dense(vocab_size, activation='softmax')(hidden_3_add)

In [14]:
model = Model([c1_inp, c2_inp, c3_inp], c4_out)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c3 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 42)         3570        c3[0][0]                         
___________________________________________________________________________________________

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [17]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=3)

Epoch 1/3
200297/200297 [==============================] - 25s - loss: 2.3976    
Epoch 2/3
200297/200297 [==============================] - 23s - loss: 2.2589    
Epoch 3/3
200297/200297 [==============================] - 24s - loss: 2.2062    


#### Test model

In [18]:
def get_next(inp):
    idxs = [char_to_idx[c] for c in inp]
    arrs = [np.array([i]) for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [19]:
get_next('phi')

'l'

In [20]:
get_next(' th')

'e'

In [21]:
get_next(' an')

'd'

## first RNN

In [37]:
clen = 8

c_in_data = [[idx[i+n] for i in range(0, len(idx)-clen-1, clen)]
             for n in range(clen)]
c_out_data = [idx[i+8] for i in range(0, len(idx)-clen-1, clen)]

In [38]:
xs = [np.array(dat) for dat in c_in_data]
y = np.array(c_out_data)

In [39]:
def embeding_input(name, n_in, n_out):
    inp = Input((1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [40]:
n_fac = 42

c_inps, c_embs = zip(*[embeding_input('c'+str(n), vocab_size, n_fac) for n in range(clen)])

dense_hidden的初始化方式是identity（单位矩阵），这样能保证在初始化时，这个dense层不会影响通过的数据（A*I=A）

原因参考Hinton的论文A Simple Way to Initialize Recurrent Networks of Rectified Linear Units：https://arxiv.org/abs/1504.00941

In [41]:
n_hidden = 256

dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [42]:
c_dens = [dense_in(c_emb) for c_emb in c_embs]

In [43]:
c_add = c_dens[0]

for n in range(1, clen):
    c_hidden = dense_hidden(c_add)
    c_add = Add()([c_hidden, c_dens[n]])

In [44]:
c_out = dense_out(c_hidden)

In [45]:
model = Model(list(c_inps), c_out)

In [46]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c0_emb (Embedding)               (None, 1, 42)         3570        c0_in[0][0]                      
____________________________________________________________________________________________________
c1_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
flatten_12 (Flatten)             (None, 42)            0           c0_emb[0][0]                     
___________________________________________________________________________________________

In [47]:
model.compile(Adam(), 'sparse_categorical_crossentropy')

In [52]:
model.fit(xs, y, epochs=5)

Epoch 1/3
75111/75111 [==============================] - 35s - loss: 2.5196    
Epoch 2/3
75111/75111 [==============================] - 33s - loss: 2.4561    
Epoch 3/3
75111/75111 [==============================] - 34s - loss: 2.4045    


#### Test model

In [53]:
get_next('for thos')

'e'

In [54]:
get_next('part of ')

't'

In [55]:
get_next('queens a')

'f'

## first RNN with Keras

In [57]:
n_fac = 42
n_hidden = 256
clen = 8

In [60]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=clen),
    SimpleRNN(n_hidden, activation='relu', recurrent_initializer='identity'),
    Dense(vocab_size, activation='softmax'),
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 42)             3570      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_11 (Dense)             (None, 85)                21845     
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(Adam(), 'sparse_categorical_crossentropy')

In [66]:
model.fit(np.concatenate(xs, axis=1), y, epochs=5)

Epoch 1/5
75111/75111 [==============================] - 39s - loss: 2.6614    
Epoch 2/5
75111/75111 [==============================] - 38s - loss: 2.1609    
Epoch 3/5
75111/75111 [==============================] - 38s - loss: 1.9578    
Epoch 4/5
75111/75111 [==============================] - 32s - loss: 1.8220    
Epoch 5/5
75111/75111 [==============================] - 36s - loss: 1.7284    


In [85]:
def get_next(s):
    idxs = [char_to_idx[c] for c in s]
    pred = model.predict(np.array([idxs]))
    pred_idx = pred.argmax(axis=1)[0]
    return idx_to_char[pred_idx]

In [86]:
get_next('this is ')

't'

In [87]:
get_next('part of ')

't'

In [88]:
get_next('queens a')

'n'

## Return Sequence

In [89]:
clen = 8

c_in_data = [[idx[i+n] for i in range(0, len(idx)-clen-1, clen)]
             for n in range(clen)]
c_out_data = [[idx[i+n+1] for i in range(0, len(idx)-clen-1, clen)]
             for n in range(clen)]

In [90]:
xs = [np.array(dat) for dat in c_in_data]
ys = [np.array(dat) for dat in c_out_data]

In [92]:
np.array(xs)[:, :5]

array([[40,  1, 33,  2, 72],
       [42,  1, 38, 44,  2],
       [29, 43, 31, 71, 54],
       [30, 45,  2, 74,  2],
       [25, 40, 73, 73, 76],
       [27, 40, 61, 61, 68],
       [29, 39, 54,  2, 66],
       [ 1, 43, 73, 62, 54]])

In [93]:
np.array(ys)[:, :5]

array([[42,  1, 38, 44,  2],
       [29, 43, 31, 71, 54],
       [30, 45,  2, 74,  2],
       [25, 40, 73, 73, 76],
       [27, 40, 61, 61, 68],
       [29, 39, 54,  2, 66],
       [ 1, 43, 73, 62, 54],
       [ 1, 33,  2, 72, 67]])

In [94]:
def embeding_input(name, n_in, n_out):
    inp = Input((1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [95]:
n_fac = 42

c_inps, c_embs = zip(*[embeding_input('c'+str(n), vocab_size, n_fac) for n in range(clen)])

In [96]:
n_hidden = 256

dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [97]:
c_dens = [dense_in(c_emb) for c_emb in c_embs]

In [100]:
c_outs = []

# 添加一个空的input(输入全为0)
inp_zeros = Input((n_fac, ), name='zeros')
c_add = dense_in(inp_zeros)
for n in range(clen):
    c_hidden = dense_hidden(c_add)
    c_add = Add()([c_hidden, c_dens[n]])
    c_outs.append(dense_out(c_add))

In [101]:
model = Model([inp_zeros]+list(c_inps), c_outs)

In [102]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
zeros (InputLayer)               (None, 42)            0                                            
____________________________________________________________________________________________________
c0_emb (Embedding)               (None, 1, 42)         3570        c0_in[0][0]                      
____________________________________________________________________________________________________
dense_12 (Dense)                 (None, 256)           11008       flatten_20[0][0]                 
                                                                   flatten_21[0][0]        

In [103]:
model.compile(Adam(), 'sparse_categorical_crossentropy')

In [106]:
zeros = np.zeros((len(xs[0]), n_fac))
zeros.shape

(75111, 42)

In [107]:
model.fit([zeros]+xs, ys, epochs=3)

Epoch 1/3
75111/75111 [==============================] - 52s - loss: 19.4717 - dense_14_loss_1: 2.6323 - dense_14_loss_2: 2.4933 - dense_14_loss_3: 2.4249 - dense_14_loss_4: 2.3949 - dense_14_loss_5: 2.3820 - dense_14_loss_6: 2.3812 - dense_14_loss_7: 2.3880 - dense_14_loss_8: 2.3751    
Epoch 2/3
75111/75111 [==============================] - 57s - loss: 17.5433 - dense_14_loss_1: 2.5067 - dense_14_loss_2: 2.3476 - dense_14_loss_3: 2.1953 - dense_14_loss_4: 2.1272 - dense_14_loss_5: 2.0962 - dense_14_loss_6: 2.0912 - dense_14_loss_7: 2.0989 - dense_14_loss_8: 2.0803    
Epoch 3/3
75111/75111 [==============================] - 52s - loss: 16.9984 - dense_14_loss_1: 2.4946 - dense_14_loss_2: 2.3303 - dense_14_loss_3: 2.1481 - dense_14_loss_4: 2.0504 - dense_14_loss_5: 2.0028 - dense_14_loss_6: 1.9938 - dense_14_loss_7: 1.9978 - dense_14_loss_8: 1.9806     ETA: 0s - loss: 17.0003 - dense_14_loss_1: 2.4951 - dense_14_loss_2: 2.3304 - dense_14_loss_3: 2.1472 - dense_14_loss_4: 2.0502 - den

In [110]:
def get_nexts(inp):
    idxs = [char_to_idx[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [111]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'c', 'n', ' ']

In [112]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'i', 'i', 'i', 'o', 'f', ' ']

## Sequence model with Keras

In [136]:
n_fac = 42
n_hidden = 256
clen = 8

理论上，Dense层是会compile错误的，因为输入有(None, 8, 256)，Dense只支持(None, x)的形式

而且TimeDistributed(Dense())和Dense()跑出来效果貌似一样

有待研究（TODO）

In [137]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=clen),
    SimpleRNN(n_hidden, activation='relu', recurrent_initializer='identity', return_sequences=True),
#     Dense(vocab_size, activation='softmax'),
    TimeDistributed(Dense(vocab_size, activation='softmax')),
])

In [138]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 8, 42)             3570      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 85)             21845     
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [139]:
model.compile(Adam(), 'sparse_categorical_crossentropy')

In [140]:
np.array(xs).shape, np.array(ys).shape

((8, 75111), (8, 75111, 1))

In [141]:
x_rnn = np.stack(xs, axis=1)
y_rnn = np.stack(ys, axis=1)
x_rnn.shape, y_rnn.shape

((75111, 8), (75111, 8, 1))

In [142]:
model.fit(x_rnn, y_rnn, epochs=3)

Epoch 1/3
75111/75111 [==============================] - 55s - loss: 2.2979    
Epoch 2/3
75111/75111 [==============================] - 54s - loss: 1.9234    
Epoch 3/3
75111/75111 [==============================] - 53s - loss: 1.8309    - ETA: 2s 


In [127]:
def get_nexts_keras(inp):
    idxs = [char_to_idx[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    print(idxs)
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [128]:
get_nexts_keras(' this is')

[2, 73, 61, 62, 72, 2, 62, 72]
[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'p', 'n', ' ']

In [132]:
pred_res = model.predict(np.array([[2, 73, 61, 62, 72, 2, 62, 72]]))

In [134]:
pred_res[:, :, :10]

array([[[  4.28957581e-09,   2.27964600e-03,   8.09430424e-03,
           7.61173396e-06,   1.00660827e-02,   1.56000067e-04,
           4.40317160e-03,   9.32861269e-07,   1.97696390e-05,
           1.23181680e-05],
        [  7.34072247e-10,   8.70438998e-06,   1.79276722e-05,
           2.12791838e-05,   1.34460433e-05,   7.82672287e-05,
           1.65134750e-09,   2.39030828e-06,   1.43394445e-05,
           1.85931719e-06],
        [  5.62891400e-10,   2.80850509e-05,   1.79209863e-04,
           1.05162171e-05,   1.24897224e-05,   1.39619697e-05,
           3.78998832e-10,   4.50970265e-06,   4.74963963e-05,
           1.28726169e-05],
        [  3.80432977e-11,   9.33941578e-07,   8.90961292e-06,
           5.22022283e-08,   2.80523892e-07,   1.11163665e-07,
           1.11883046e-11,   7.10334191e-08,   2.68941199e-06,
           2.67958103e-06],
        [  1.12557341e-08,   1.41721725e-01,   7.61174262e-01,
           6.65230618e-04,   4.90249135e-04,   2.97799088e-05,
      

## LSTM

In [149]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=clen, batch_input_shape=(64,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (64, 8, 42)               3570      
_________________________________________________________________
batch_normalization_2 (Batch (64, 8, 42)               168       
_________________________________________________________________
lstm_1 (LSTM)                (64, 8, 256)              306176    
_________________________________________________________________
time_distributed_2 (TimeDist (64, 8, 85)               21845     
Total params: 331,759
Trainable params: 331,675
Non-trainable params: 84
_________________________________________________________________
